# Page rank

## Imports

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd

import networkx as nx
import graphviz as gv

## Network classes

In [3]:
class Page:
    """A class representing a webpage."""
    def __init__(self, name):
        """Constructor method, takes the name of the webpage as an argument."""
        self.name = name
        self.outgoingLinks = []
        self.incomingLinks = []
        
    def __repr__(self):
        """A page is represented by its name."""
        return self.name
    
    def addLinkFrom(self, other):
        """Adds a link from another page, ending at this page"""
        self.incomingLinks.append(other)
    
    def addLinkTo(self, other):
        """Adds a link to another page, starting from this page"""
        self.outgoingLinks.append(other)
        other.addLinkFrom(self)
        
    def addLinksTo(self, *args):
        """Adds a link to another page, starting from this page"""
        for page in args:
            self.addLinkTo(page)
        
    def outDegree(self):
        """Returns the amount of links this page has to other pages, i.e. its out-degree"""
        return len(self.outgoingLinks)
    
    def getName(self):
        return self.name
    
    def linksTo(self, other) -> bool:
        return other in self.outgoingLinks

In [4]:
class Network:
    """A class representing a network of webpages"""
    def __init__(self):
        """A network starts without any pages."""
        self.pages = []
        self.counter = 0
        
        # Whether the dataframe is out of date
        self.old_df = True
    
    def __iter__(self):
        """Iterating over a network is the same as iterating over its pages."""
        return iter(self.pages)
        
    def addPage(self, page: Page):
        """Adds a page to the network"""
        # give the page an id (since the name of the webpage doesn't have to be unique)
        page.id = self.counter
        self.counter += 1
        self.pages.append(page)
        
        # Dataframe is out of date now
        self.old_df = True
        
    def addPages(self, *args):
        """Adds multiple pages to the network"""
        for page in args:
            self.addPage(page)
        
    def getPages(self):
        """Returns the pages in the network."""
        return self.pages
    
    def size(self):
        """Returns the size of the network."""
        return len(self.pages)
    
    def showRanking(self):
        """
        Prints the pages with their corresponding rank. 
        Not sorted on anything in particular, prints in increasing id.
        """
        for page in self:
            print(f'{page.name}: {page.rank}')
            
    def debugRanking(self):
        """
        Computes the sum of the rank of all pages in the network, which *should* return 1.
        """
        rankSum = 0
        for page in self:
            rankSum += page.rank
        print(rankSum)
    
    # ----------------------------------------------------------
    # --------------------- VISUALIZATION ----------------------
    # ----------------------------------------------------------
    
    # ------------------- NODE-LINK DIAGRAMS ------------------- 
    def nxDiGraph(self):
        """Calculates networkx DiGraph"""
        # make a directed graph
        G = nx.DiGraph()
        
        # add all the edges and nodes to it
        for page in self:
            G.add_node(page)
            for other in page.outgoingLinks:
                G.add_edge(page, other)
        
        return G
    
    def plot1(self, names=False):
        """
        Plot using networkx.
        Node size based on *number of links*
        """
        G = self.nxDiGraph()
        
        # calculate a layout (force-directed)
        pos = nx.layout.spring_layout(G)

        # scale nodes by amount of links
        node_sizes = [1000 + 3000*len(page.outgoingLinks) for page in G.nodes()]
        
        if not names:
            # labels
            labels = {page:page.id for page in G.nodes()}
            
            nx.draw(G, pos, with_labels=True, labels=labels, node_size=node_sizes, arrowsize=40, font_color="white")
        else:
            nx.draw(G, pos, with_labels=True, node_size=node_sizes, arrowsize=40, font_color="white")
        plt.show()
    
    def plot2(self, names=False):
        """Plot using networkx. Node size based on *rank*"""
        G = self.nxDiGraph()
        
        # calculate a layout (force-directed)
        pos = nx.layout.spring_layout(G)

        # scale nodes by rank
        node_sizes = [1000 + 3000*page.rank for page in G.nodes()]
        
        if not names:
            # labels
            labels = {page:page.id for page in G.nodes()}
            
            nx.draw(G, pos, with_labels=True, labels=labels, node_size=node_sizes, arrowsize=40, font_color="white")
        else:
            nx.draw(G, pos, with_labels=True, node_size=node_sizes, arrowsize=40, font_color="white")
        plt.show()
        
    def plot0(self):
        """Plot using graphviz"""
        f = gv.Digraph()
        for page in self:
            f.node(str(page))
            for other in page.outgoingLinks:
                f.edge(str(page), str(other))

        return f
    
    # ----------------------- STATISTICS -----------------------
    def updateDf(self):
        """Update pandas DataFrame of this network."""
        
        # Prepare a dataframe
        df = pd.DataFrame(columns=['Name', 'Outdegree', 'Indegree'])
        
        df['Name'] = list(map(Page.getName, self.pages))
        df['Outdegree'] = list(map(Page.outDegree, self.pages))
        df.fillna(0, inplace=True)
        
        # Count indegree of every page
        for page in self:
            for link in page.outgoingLinks:
                df.at[link.id, 'Indegree'] += 1
        
        # Update dataframe
        self.df = df
        self.old_df = False
        
    def plot_outdegrees(self):
        """Plots a histogram of the outdegree of the pages in the network."""
        if self.old_df:
            self.updateDf()
        
        ax = sns.distplot(self.df['Outdegree'], kde=False, rug=True)
        ax.set_xlabel("Outdegree (number of links to other pages)")
        ax.set_ylabel("Frequency")
        
    def plot_indegrees(self):
        """Plots a histogram of the indegree of the pages in the network."""
        if self.old_df:
            self.updateDf()
        
        ax = sns.distplot(self.df['Indegree'], kde=False, rug=True)
        ax.set_xlabel("Indegree (number of links to this page from other pages)")
        ax.set_ylabel("Frequency")
        
    def describe(self):
        """Prints a table of summary statistics of this network."""
        if self.old_df:
            self.updateDf()
            
        return self.df.describe()
    
    def getDf(self):
        """Returns the dataframe of this network, updates it when needed."""
        if self.old_df:
            self.updateDf()
            
        return self.df
    
    def sinks(self):
        """Returns the sinks of the network."""
        if self.old_df:
            self.updateDf()
            
        return self.df[self.df['Outdegree'] == 0]
        
    def sources(self):
        """Returns the sources of the network."""
        if self.old_df:
            self.updateDf()
            
        return self.df[self.df['Indegree'] == 0]

### Example Wikipedia-YouTube-Twitter

In [23]:
wiki = Page("Wikipedia")
yt = Page("YouTube")
twitter = Page("Twitter")

wiki.addLinkTo(yt)
wiki.addLinkTo(twitter)
yt.addLinkTo(twitter)
twitter.addLinkTo(yt)

In [24]:
internet = Network()
internet.addPages(yt, wiki, twitter)

In [25]:
pages = internet.getPages()
pages

[YouTube, Wikipedia, Twitter]

In [26]:
yt.incomingLinks

[Wikipedia, Twitter]

In [27]:
yt.outgoingLinks

[Twitter]

### Example California Network

In [ ]:
with open("california.txt") as f:
    content = f.read().splitlines()

In [ ]:
california = Network()
for line in content:
    if line[0] == 'n':
        california.addPage(Page(line.split()[2]))
    elif line[0] == 'e':
        n1 = california.pages[int(line.split()[1])]
        n2 = california.pages[int(line.split()[2])]
        
        n1.addLinkTo(n2)

## Ranking the pages

In [5]:
class PageRanker:
    """A static class for ranking a network."""
    
    @staticmethod
    def rankCR(network: Network, steps: int, alpha: float): # rankCR = rankCertainRestart
        """
        Ranks a network via the PageRank algorithm.
        
        --- Parameters ---
        network: The Network to rank.
        steps: The number of iterations to do.
        alpha: The probability of going to another page via clicking a link.
        """
        PageRanker.calculateInitialRanking(network)
        PageRanker.calculateJumpingProbabilities(network)
        
        newRanks = {}
        for _ in range(0, steps):
#             text = "("
#             for page in network:
#                 text += str(page.rank)
#                 text += ", "
#             text += ")"
#             print(text)
#             network.debugRanking()

            for page in network:
                newRanks[page] = 0
            # Sum dot product of column and stationary distribution
                for other in network:
                    if other.outDegree() > 0:
                        newRanks[page] += other.rank * page.q * (1 - alpha)
                    else:
                        newRanks[page] += other.rank * page.q
                    if other.linksTo(page):
                        newRanks[page] += other.rank * alpha / other.outDegree()

            for page in network:
                page.rank = newRanks[page]
    
    @staticmethod
    def rank(network: Network, steps: int, alpha: float):         # Runtime O(steps * #pages * max. incoming links for a page)
        """
        Ranks a network via the PageRank algorithm.
        
        --- Parameters ---
        network: The Network to rank.
        steps: The number of iterations to do.
        alpha: The probability of going to another page via clicking a link.
        """
        PageRanker.calculateInitialRanking(network)
        PageRanker.calculateJumpingProbabilities(network)
        
        newRanks = {}
        for _ in range(0, steps):
#             text = "("
#             for page in network:
#                 text += str(page.rank)
#                 text += ", "
#             text += ")"
#             print(text)
#             network.debugRanking()

            for page in network:
                newRanks[page] = 0
                for other in page.incomingLinks:
                    newRanks[page] += other.rank / other.outDegree()
                newRanks[page] *= alpha
                newRanks[page] += (1-alpha) * page.q
                
            for page in network:
                page.rank = newRanks[page]

        # Normalize newRanks vector so it sums to 1 & declare new ranks
        # normalize at the end, but not every step -- dan geeft het dezelfde resultaten als networkx.
        # Weet niet of dat beter is maar lijkt me wel? Anders probeert een rank te convergeren naar een
        # waarde en wordt het steeds terug genormalizeerd.
        newRanksSum = sum(newRanks.values())
        for page in network:
            page.rank = newRanks[page]/newRanksSum

    
    @staticmethod
    def calculateInitialRanking(network: Network):
        """
        Calculates an initial ranking for the given network.
        Currently just gives each page a ranking of 1 / size of the network.
        """
        for page in network:
            page.rank = 1.0 / network.size()
#             print("initial rank for " + str(network.size()) + " pages is " + str((1.0 / network.size())) )
            # When looking at the above print command for large networks, the sum doesn't necessarily add to 1 perfectly
            # anymore due to rounding errors
            
    @staticmethod
    def calculateJumpingProbabilities(network: Network):
        """Calculates the probabilities of jumping to each page."""
        for page in network:
            page.q = 1.0 / network.size()

### Example Wikipedia-YouTube-Twitter Calculation

#### Wikipedia-YouTube-Twitter with the Certain Restart Version

In [28]:
PageRanker.rankCR(internet, 10, 0.5)
internet.showRanking()
internet.debugRanking()

YouTube: 0.41666666666666663
Wikipedia: 0.16666666666666663
Twitter: 0.41666666666666663
0.9999999999999999


As long as the network does not have a sink, the ranks seem to be calculated correctly and still sum to 1. 

#### Wikipedia-YouTube-Twitter with Normalizer Version

In [29]:
PageRanker.rank(internet, 10, 0.5) 

In [30]:
internet.showRanking()

YouTube: 0.4166666666666667
Wikipedia: 0.16666666666666669
Twitter: 0.4166666666666667


In [31]:
internet.debugRanking() # This one is the one with the normalizer feature, naturally it sums perfectly to 1 for this simple network

1.0


### Example California Network Calculation

#### Certain Restart Version

In [ ]:
# slow (this is only 1 iteration!)
# PageRanker.rankCR(california, 1, 0.5)
# california.showRanking()
# california.debugRanking()

#### Normalizer Version

In [ ]:
PageRanker.rank(california, 100, 0.5)
california.showRanking()
california.debugRanking()

### Edge Case: Source -> Sink

#### Normalizer Version

In [ ]:
criminal = Network()
page1 = Page("Source")
page2 = Page("Sink")
page1.addLinkTo(page2)
criminal.addPages(page1, page2)

In [ ]:
criminal.plot0()

In [ ]:
PageRanker.rank(criminal, 10, 0.5)
criminal.showRanking()
criminal.debugRanking()

In [ ]:
nx.pagerank(criminal.nxDiGraph(), alpha=0.5, max_iter=10)

#### Certain Restart Version

In [ ]:
PageRanker.rankCR(criminal, 10, 0.5)
criminal.showRanking()
criminal.debugRanking()

### Timo's handsolved network

In [ ]:
handsolved = Network()
w1 = Page("Website 1")
w2 = Page("Website 2")
w3 = Page("Website 3")
w4 = Page("Website 4")
w5 = Page("Website 5")

w1.addLinksTo(w2, w3, w4, w5)
w2.addLinksTo(w3, w4, w5)
w3.addLinksTo(w1, w2, w4, w5)
w4.addLinksTo(w2, w3)
w5.addLinkTo(w4)

handsolved.addPages(w1, w2, w3, w4, w5)
handsolved.plot0()

In [ ]:
PageRanker.rankCR(handsolved, 100, 0.5)
handsolved.showRanking()

In [ ]:
print(f'Website 1: {36/283}')
print(f'Website 2: {297/1415}')
print(f'Website 3: {308/1415}')
print(f'Website 4: {378/1415}')
print(f'Website 5: {252/1415}')

## Visualization of (small) networks

### With networkx

In [ ]:
# size based on links
internet.plot1()

In [ ]:
# plot with names (works in this case, but often labels are too large for the nodes)
internet.plot1(names=True)

In [ ]:
#size based on rank
internet.plot2()

### With graphviz

In [ ]:
#internet.plot0()

### Example Calculation with Homepage to 99 Sites

#### Certain Restart Version

In [ ]:
internetV2= Network()
homepage = Page("Homepage")
for i in range(0,99):
    p = Page("test" + str(i))
    homepage.addLinkTo(p)
    internetV2.addPage(p)
internetV2.addPage(homepage)
PageRanker.rankCR(internetV2, 100, 0.5)
internetV2.showRanking()
internetV2.debugRanking()

#### Normalizer Version

In [ ]:
PageRanker.rank(internetV2, 100, 0.5)
internetV2.showRanking()
internetV2.debugRanking()

In [ ]:
internetV2.plot2()

### Example Calculation with "Endpage" which is linked to from 99 Sites

#### Certain Restart Version

In [ ]:
internetV3= Network()
endpage = Page("Endpage")
for i in range(0,99):
    p = Page("test" + str(i))
    p.addLinkTo(endpage)
    internetV3.addPage(p)
internetV3.addPage(endpage)
PageRanker.rankCR(internetV3, 1000, 0.5)
internetV3.showRanking()
internetV3.debugRanking()

#### Normalizer Version

In [ ]:
PageRanker.rank(internetV3, 1000, 0.5)
internetV3.showRanking()
internetV3.debugRanking()

## Statistics

In [ ]:
internet.sources()

In [ ]:
internet.sinks()

In [ ]:
california.sinks()

In [ ]:
california.sources()

In [ ]:
internet.plot_outdegrees()

In [ ]:
internet.plot_indegrees()

In [ ]:
internet.describe()

In [ ]:
california.plot_outdegrees()

In [ ]:
california.plot_indegrees()

In [ ]:
california.describe()

# Example networks

## 10 websites

### Twee of drie verschillende stelsels, Custimization: Add third little network of 2 websites and/or add a link from website 8 to these two websites

In [59]:
#hier kunnen zo'n hashtags voor worden gezet voor verschillende stelsels/links

Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")

Website1.addLinkTo(Website2)
Website1.addLinkTo(Website3)
Website1.addLinkTo(Website4)

Website2.addLinkTo(Website1)
Website2.addLinkTo(Website3)
Website2.addLinkTo(Website4)

Website3.addLinkTo(Website1)
Website3.addLinkTo(Website2)
Website3.addLinkTo(Website4)

Website4.addLinkTo(Website1)
Website4.addLinkTo(Website2)
Website4.addLinkTo(Website3)

Website5.addLinkTo(Website6)
Website5.addLinkTo(Website7)
Website5.addLinkTo(Website8)

Website6.addLinkTo(Website5)
Website6.addLinkTo(Website7)
Website6.addLinkTo(Website8)

Website7.addLinkTo(Website5)
Website7.addLinkTo(Website6)
Website7.addLinkTo(Website8)

Website8.addLinkTo(Website5)
Website8.addLinkTo(Website6)
Website8.addLinkTo(Website7)

#Website8.addLinkTo(Website9)
#Website8.addLinkTo(Website10)

#Website9.addLinkTo(Website10)
#Website10.addLinkTo(Website9)


network2Different = Network()
network2Different.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8)
#network2Different.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10)

In [60]:
PageRanker.rankCR(network2Different, 10, 0.5)
network2Different.showRanking()
network2Different.debugRanking()

Website1: 0.12499999999999999
Website2: 0.12499999999999999
Website3: 0.12499999999999999
Website4: 0.12499999999999999
Website5: 0.12499999999999999
Website6: 0.12499999999999999
Website7: 0.12499999999999999
Website8: 0.12499999999999999
0.9999999999999999


In [61]:
PageRanker.rank(network2Different, 10, 0.5)
network2Different.showRanking()

Website1: 0.125
Website2: 0.125
Website3: 0.125
Website4: 0.125
Website5: 0.125
Website6: 0.125
Website7: 0.125
Website8: 0.125


In [36]:
network2Different.debugRanking()

1.0


### Everything to website 1, Custimization: add links from website 1 to all pages

In [46]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")

#Website1.addLinkTo(Website2)
#Website1.addLinkTo(Website3)
#Website1.addLinkTo(Website4)
#Website1.addLinkTo(Website5)
#Website1.addLinkTo(Website6)
#Website1.addLinkTo(Website7)
#Website1.addLinkTo(Website8)
#Website1.addLinkTo(Website9)
#Website1.addLinkTo(Website10)


Website2.addLinkTo(Website1)
Website3.addLinkTo(Website1)
Website4.addLinkTo(Website1)
Website5.addLinkTo(Website1)
Website6.addLinkTo(Website1)
Website7.addLinkTo(Website1)
Website8.addLinkTo(Website1)
Website9.addLinkTo(Website1)
Website10.addLinkTo(Website1)


networkAllToOne = Network()
networkAllToOne.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10)

In [47]:
PageRanker.rankCR(networkAllToOne, 10, 0.5)
networkAllToOne.showRanking()
networkAllToOne.debugRanking()

Website1: 0.3792152378985449
Website2: 0.06897608467793949
Website3: 0.06897608467793949
Website4: 0.06897608467793949
Website5: 0.06897608467793949
Website6: 0.06897608467793949
Website7: 0.06897608467793949
Website8: 0.06897608467793949
Website9: 0.06897608467793949
Website10: 0.06897608467793949
1.0000000000000002


In [48]:
PageRanker.rank(networkAllToOne, 10, 0.5)
networkAllToOne.showRanking()
networkAllToOne.debugRanking()

Website1: 0.37931034482758613
Website2: 0.06896551724137931
Website3: 0.06896551724137931
Website4: 0.06896551724137931
Website5: 0.06896551724137931
Website6: 0.06896551724137931
Website7: 0.06896551724137931
Website8: 0.06896551724137931
Website9: 0.06896551724137931
Website10: 0.06896551724137931
1.0


### All websites go to eachother except website 10

In [111]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")

Website1.addLinkTo(Website2)
Website1.addLinkTo(Website3)
Website1.addLinkTo(Website4)
Website1.addLinkTo(Website5)
Website1.addLinkTo(Website6)
Website1.addLinkTo(Website7)
Website1.addLinkTo(Website8)
Website1.addLinkTo(Website9)
Website1.addLinkTo(Website10)

Website2.addLinkTo(Website1)
Website2.addLinkTo(Website3)
Website2.addLinkTo(Website4)
Website2.addLinkTo(Website5)
Website2.addLinkTo(Website6)
Website2.addLinkTo(Website7)
Website2.addLinkTo(Website8)
Website2.addLinkTo(Website9)
Website2.addLinkTo(Website10)

Website3.addLinkTo(Website1)
Website3.addLinkTo(Website2)
Website3.addLinkTo(Website4)
Website3.addLinkTo(Website5)
Website3.addLinkTo(Website6)
Website3.addLinkTo(Website7)
Website3.addLinkTo(Website8)
Website3.addLinkTo(Website9)
Website3.addLinkTo(Website10)

Website4.addLinkTo(Website1)
Website4.addLinkTo(Website2)
Website4.addLinkTo(Website3)
Website4.addLinkTo(Website5)
Website4.addLinkTo(Website6)
Website4.addLinkTo(Website7)
Website4.addLinkTo(Website8)
Website4.addLinkTo(Website9)
Website4.addLinkTo(Website10)

Website5.addLinkTo(Website1)
Website5.addLinkTo(Website2)
Website5.addLinkTo(Website3)
Website5.addLinkTo(Website4)
Website5.addLinkTo(Website6)
Website5.addLinkTo(Website7)
Website5.addLinkTo(Website8)
Website5.addLinkTo(Website9)
Website5.addLinkTo(Website10)

Website6.addLinkTo(Website1)
Website6.addLinkTo(Website2)
Website6.addLinkTo(Website3)
Website6.addLinkTo(Website4)
Website6.addLinkTo(Website5)
Website6.addLinkTo(Website7)
Website6.addLinkTo(Website8)
Website6.addLinkTo(Website9)
Website6.addLinkTo(Website10)

Website7.addLinkTo(Website1)
Website7.addLinkTo(Website2)
Website7.addLinkTo(Website3)
Website7.addLinkTo(Website4)
Website7.addLinkTo(Website5)
Website7.addLinkTo(Website6)
Website7.addLinkTo(Website8)
Website7.addLinkTo(Website9)
Website7.addLinkTo(Website10)

Website8.addLinkTo(Website1)
Website8.addLinkTo(Website2)
Website8.addLinkTo(Website3)
Website8.addLinkTo(Website4)
Website8.addLinkTo(Website5)
Website8.addLinkTo(Website6)
Website8.addLinkTo(Website7)
Website8.addLinkTo(Website9)
Website8.addLinkTo(Website10)

Website9.addLinkTo(Website1)
Website9.addLinkTo(Website2)
Website9.addLinkTo(Website3)
Website9.addLinkTo(Website4)
Website9.addLinkTo(Website5)
Website9.addLinkTo(Website6)
Website9.addLinkTo(Website7)
Website9.addLinkTo(Website8)
Website9.addLinkTo(Website10)

networkAllButOneToAll = Network()
networkAllButOneToAll.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10)

In [112]:
PageRanker.rankCR(networkAllButOneToAll, 10, 0.5)
networkAllButOneToAll.showRanking()
networkAllButOneToAll.debugRanking()

Website1: 0.09944751381215473
Website2: 0.09944751381215473
Website3: 0.09944751381215473
Website4: 0.09944751381215473
Website5: 0.09944751381215473
Website6: 0.09944751381215473
Website7: 0.09944751381215473
Website8: 0.09944751381215473
Website9: 0.09944751381215473
Website10: 0.10497237569060777
1.0000000000000004


In [113]:
PageRanker.rank(networkAllButOneToAll, 10, 0.5)
networkAllButOneToAll.showRanking()
networkAllButOneToAll.debugRanking()

Website1: 0.09944749086424848
Website2: 0.09944749086424848
Website3: 0.09944749086424848
Website4: 0.09944749086424848
Website5: 0.09944749086424848
Website6: 0.09944749086424848
Website7: 0.09944749086424848
Website8: 0.09944749086424848
Website9: 0.09944749086424848
Website10: 0.10497258222176362
0.9999999999999999


### Circle network

In [62]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")

Website1.addLinkTo(Website2)
Website2.addLinkTo(Website3)
Website3.addLinkTo(Website4)
Website4.addLinkTo(Website5)
Website5.addLinkTo(Website6)
Website6.addLinkTo(Website7)
Website7.addLinkTo(Website8)
Website8.addLinkTo(Website9)
Website9.addLinkTo(Website10)
Website10.addLinkTo(Website1)

networkCircle = Network()
networkCircle.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10)

In [63]:
PageRanker.rankCR(networkCircle, 10, 0.5)
networkCircle.showRanking()
networkCircle.debugRanking()

Website1: 0.10000000000000009
Website2: 0.10000000000000009
Website3: 0.10000000000000009
Website4: 0.10000000000000009
Website5: 0.10000000000000009
Website6: 0.10000000000000009
Website7: 0.10000000000000009
Website8: 0.10000000000000009
Website9: 0.10000000000000009
Website10: 0.10000000000000009
1.0000000000000009


In [64]:
PageRanker.rank(networkCircle, 10, 0.5)
networkCircle.showRanking()
networkCircle.debugRanking()

Website1: 0.10000000000000002
Website2: 0.10000000000000002
Website3: 0.10000000000000002
Website4: 0.10000000000000002
Website5: 0.10000000000000002
Website6: 0.10000000000000002
Website7: 0.10000000000000002
Website8: 0.10000000000000002
Website9: 0.10000000000000002
Website10: 0.10000000000000002
1.0


### Two clusters with one link

In [65]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")

Website1.addLinkTo(Website2)
Website1.addLinkTo(Website3)
Website1.addLinkTo(Website4)
Website1.addLinkTo(Website5)

Website2.addLinkTo(Website1)
Website2.addLinkTo(Website3)
Website2.addLinkTo(Website4)
Website2.addLinkTo(Website5)

Website3.addLinkTo(Website1)
Website3.addLinkTo(Website2)
Website3.addLinkTo(Website4)
Website3.addLinkTo(Website5)

Website4.addLinkTo(Website1)
Website4.addLinkTo(Website2)
Website4.addLinkTo(Website3)
Website4.addLinkTo(Website5)

Website5.addLinkTo(Website1)
Website5.addLinkTo(Website2)
Website5.addLinkTo(Website3)
Website5.addLinkTo(Website4)
Website5.addLinkTo(Website6)

Website6.addLinkTo(Website5)
Website6.addLinkTo(Website7)
Website6.addLinkTo(Website8)
Website6.addLinkTo(Website9)
Website6.addLinkTo(Website10)

Website7.addLinkTo(Website6)
Website7.addLinkTo(Website8)
Website7.addLinkTo(Website9)
Website7.addLinkTo(Website10)

Website8.addLinkTo(Website6)
Website8.addLinkTo(Website7)
Website8.addLinkTo(Website9)
Website8.addLinkTo(Website10)

Website9.addLinkTo(Website6)
Website9.addLinkTo(Website7)
Website9.addLinkTo(Website8)
Website9.addLinkTo(Website10)

Website10.addLinkTo(Website6)
Website10.addLinkTo(Website7)
Website10.addLinkTo(Website8)
Website10.addLinkTo(Website9)


networkCluster = Network()
networkCluster.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10)

In [66]:
PageRanker.rankCR(networkCluster, 10, 0.5)
networkCluster.showRanking()
networkCluster.debugRanking()

Website1: 0.09756097560975614
Website2: 0.09756097560975614
Website3: 0.09756097560975614
Website4: 0.09756097560975614
Website5: 0.10975609756097565
Website6: 0.10975609756097564
Website7: 0.09756097560975613
Website8: 0.09756097560975613
Website9: 0.09756097560975613
Website10: 0.09756097560975613
1.0000000000000004


In [71]:
PageRanker.rank(networkCluster, 10, 0.5)
networkCluster.showRanking()
networkCluster.debugRanking()

Website1: 0.09756097560975611
Website2: 0.09756097560975611
Website3: 0.09756097560975611
Website4: 0.09756097560975611
Website5: 0.10975609756097564
Website6: 0.10975609756097562
Website7: 0.09756097560975611
Website8: 0.09756097560975611
Website9: 0.09756097560975611
Website10: 0.09756097560975611
1.0


## Now the same networks but 100 websites

### All websites link to one website

In [82]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")
Website11 = Page("Website11")
Website12 = Page("Website12")
Website13 = Page("Website13")
Website14 = Page("Website14")
Website15 = Page("Website15")
Website16 = Page("Website16")
Website17 = Page("Website17")
Website18 = Page("Website18")
Website19 = Page("Website19")
Website20 = Page("Website20")
Website21 = Page("Website21")
Website22 = Page("Website22")
Website23 = Page("Website23")
Website24 = Page("Website24")
Website25 = Page("Website25")
Website26 = Page("Website26")
Website27 = Page("Website27")
Website28 = Page("Website28")
Website29 = Page("Website29")
Website30 = Page("Website30")
Website31 = Page("Website31")
Website32 = Page("Website32")
Website33 = Page("Website33")
Website34 = Page("Website34")
Website35 = Page("Website35")
Website36 = Page("Website36")
Website37 = Page("Website37")
Website38 = Page("Website38")
Website39 = Page("Website39")
Website40 = Page("Website40")
Website41 = Page("Website41")
Website42 = Page("Website42")
Website43 = Page("Website43")
Website44 = Page("Website44")
Website45 = Page("Website45")
Website46 = Page("Website46")
Website47 = Page("Website47")
Website48 = Page("Website48")
Website49 = Page("Website49")
Website50 = Page("Website50")
Website51 = Page("Website51")
Website52 = Page("Website52")
Website53 = Page("Website53")
Website54 = Page("Website54")
Website55 = Page("Website55")
Website56 = Page("Website56")
Website57 = Page("Website57")
Website58 = Page("Website58")
Website59 = Page("Website59")
Website60 = Page("Website60")
Website61 = Page("Website61")
Website62 = Page("Website62")
Website63 = Page("Website63")
Website64 = Page("Website64")
Website65 = Page("Website65")
Website66 = Page("Website66")
Website67 = Page("Website67")
Website68 = Page("Website68")
Website69 = Page("Website69")
Website70 = Page("Website70")
Website71 = Page("Website71")
Website72 = Page("Website72")
Website73 = Page("Website73")
Website74 = Page("Website74")
Website75 = Page("Website75")
Website76 = Page("Website76")
Website77 = Page("Website77")
Website78 = Page("Website78")
Website79 = Page("Website79")
Website80 = Page("Website80")
Website81 = Page("Website81")
Website82 = Page("Website82")
Website83 = Page("Website83")
Website84 = Page("Website84")
Website85 = Page("Website85")
Website86 = Page("Website86")
Website87 = Page("Website87")
Website88 = Page("Website88")
Website89 = Page("Website89")
Website90 = Page("Website90")
Website91 = Page("Website91")
Website92 = Page("Website92")
Website93 = Page("Website93")
Website94 = Page("Website94")
Website95 = Page("Website95")
Website96 = Page("Website96")
Website97 = Page("Website97")
Website98 = Page("Website98")
Website99 = Page("Website99")
Website100 = Page("Website100")

WebsiteList = [Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
               Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
               Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
               Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
               Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
               Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
               Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
               Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
               Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
               Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100]
               
i=1
while i<100:
    Website1.addLinkTo(WebsiteList[i])
    WebsiteList[i].addLinkTo(Website1)
    i+=1
    
networkAllToOne100 = Network()
networkAllToOne100.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
                           Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
                           Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
                           Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
                           Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
                           Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
                           Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
                           Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
                           Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
                           Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100)

In [87]:
PageRanker.rankCR(networkAllToOne100, 100, 0.5)
networkAllToOne100.showRanking()
networkAllToOne100.debugRanking()

Website1: 0.3366666666666646
Website2: 0.006700336700336652
Website3: 0.006700336700336652
Website4: 0.006700336700336652
Website5: 0.006700336700336652
Website6: 0.006700336700336652
Website7: 0.006700336700336652
Website8: 0.006700336700336652
Website9: 0.006700336700336652
Website10: 0.006700336700336652
Website11: 0.006700336700336652
Website12: 0.006700336700336652
Website13: 0.006700336700336652
Website14: 0.006700336700336652
Website15: 0.006700336700336652
Website16: 0.006700336700336652
Website17: 0.006700336700336652
Website18: 0.006700336700336652
Website19: 0.006700336700336652
Website20: 0.006700336700336652
Website21: 0.006700336700336652
Website22: 0.006700336700336652
Website23: 0.006700336700336652
Website24: 0.006700336700336652
Website25: 0.006700336700336652
Website26: 0.006700336700336652
Website27: 0.006700336700336652
Website28: 0.006700336700336652
Website29: 0.006700336700336652
Website30: 0.006700336700336652
Website31: 0.006700336700336652
Website32: 0.006700

In [88]:
PageRanker.rank(networkAllToOne100, 10, 0.5)
networkAllToOne100.showRanking()
networkAllToOne100.debugRanking()

Website1: 0.33634765625000074
Website2: 0.006703559027777785
Website3: 0.006703559027777785
Website4: 0.006703559027777785
Website5: 0.006703559027777785
Website6: 0.006703559027777785
Website7: 0.006703559027777785
Website8: 0.006703559027777785
Website9: 0.006703559027777785
Website10: 0.006703559027777785
Website11: 0.006703559027777785
Website12: 0.006703559027777785
Website13: 0.006703559027777785
Website14: 0.006703559027777785
Website15: 0.006703559027777785
Website16: 0.006703559027777785
Website17: 0.006703559027777785
Website18: 0.006703559027777785
Website19: 0.006703559027777785
Website20: 0.006703559027777785
Website21: 0.006703559027777785
Website22: 0.006703559027777785
Website23: 0.006703559027777785
Website24: 0.006703559027777785
Website25: 0.006703559027777785
Website26: 0.006703559027777785
Website27: 0.006703559027777785
Website28: 0.006703559027777785
Website29: 0.006703559027777785
Website30: 0.006703559027777785
Website31: 0.006703559027777785
Website32: 0.00670

### All websites go to eachother except website 1

In [130]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")
Website11 = Page("Website11")
Website12 = Page("Website12")
Website13 = Page("Website13")
Website14 = Page("Website14")
Website15 = Page("Website15")
Website16 = Page("Website16")
Website17 = Page("Website17")
Website18 = Page("Website18")
Website19 = Page("Website19")
Website20 = Page("Website20")
Website21 = Page("Website21")
Website22 = Page("Website22")
Website23 = Page("Website23")
Website24 = Page("Website24")
Website25 = Page("Website25")
Website26 = Page("Website26")
Website27 = Page("Website27")
Website28 = Page("Website28")
Website29 = Page("Website29")
Website30 = Page("Website30")
Website31 = Page("Website31")
Website32 = Page("Website32")
Website33 = Page("Website33")
Website34 = Page("Website34")
Website35 = Page("Website35")
Website36 = Page("Website36")
Website37 = Page("Website37")
Website38 = Page("Website38")
Website39 = Page("Website39")
Website40 = Page("Website40")
Website41 = Page("Website41")
Website42 = Page("Website42")
Website43 = Page("Website43")
Website44 = Page("Website44")
Website45 = Page("Website45")
Website46 = Page("Website46")
Website47 = Page("Website47")
Website48 = Page("Website48")
Website49 = Page("Website49")
Website50 = Page("Website50")
Website51 = Page("Website51")
Website52 = Page("Website52")
Website53 = Page("Website53")
Website54 = Page("Website54")
Website55 = Page("Website55")
Website56 = Page("Website56")
Website57 = Page("Website57")
Website58 = Page("Website58")
Website59 = Page("Website59")
Website60 = Page("Website60")
Website61 = Page("Website61")
Website62 = Page("Website62")
Website63 = Page("Website63")
Website64 = Page("Website64")
Website65 = Page("Website65")
Website66 = Page("Website66")
Website67 = Page("Website67")
Website68 = Page("Website68")
Website69 = Page("Website69")
Website70 = Page("Website70")
Website71 = Page("Website71")
Website72 = Page("Website72")
Website73 = Page("Website73")
Website74 = Page("Website74")
Website75 = Page("Website75")
Website76 = Page("Website76")
Website77 = Page("Website77")
Website78 = Page("Website78")
Website79 = Page("Website79")
Website80 = Page("Website80")
Website81 = Page("Website81")
Website82 = Page("Website82")
Website83 = Page("Website83")
Website84 = Page("Website84")
Website85 = Page("Website85")
Website86 = Page("Website86")
Website87 = Page("Website87")
Website88 = Page("Website88")
Website89 = Page("Website89")
Website90 = Page("Website90")
Website91 = Page("Website91")
Website92 = Page("Website92")
Website93 = Page("Website93")
Website94 = Page("Website94")
Website95 = Page("Website95")
Website96 = Page("Website96")
Website97 = Page("Website97")
Website98 = Page("Website98")
Website99 = Page("Website99")
Website100 = Page("Website100")

WebsiteList = [Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
               Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
               Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
               Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
               Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
               Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
               Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
               Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
               Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
               Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100]
               
i=1
j=0
while i<100:
    while j<100:
        if i != j:
            WebsiteList[i].addLinkTo(WebsiteList[j])
        j+=1
    j=0
    i+=1
    
networkAllButOneToAll100 = Network()
networkAllButOneToAll100.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
                           Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
                           Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
                           Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
                           Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
                           Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
                           Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
                           Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
                           Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
                           Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100)

In [132]:
Website1.outgoingLinks

[]

In [133]:
PageRanker.rankCR(networkAllButOneToAll100, 100, 0.5)
networkAllButOneToAll100.showRanking()
networkAllButOneToAll100.debugRanking()

Website1: 0.010049997474875037
Website2: 0.009999494975001293
Website3: 0.009999494975001293
Website4: 0.009999494975001293
Website5: 0.009999494975001293
Website6: 0.009999494975001293
Website7: 0.009999494975001293
Website8: 0.009999494975001293
Website9: 0.009999494975001293
Website10: 0.009999494975001293
Website11: 0.009999494975001293
Website12: 0.009999494975001293
Website13: 0.009999494975001293
Website14: 0.009999494975001293
Website15: 0.009999494975001293
Website16: 0.009999494975001293
Website17: 0.009999494975001293
Website18: 0.009999494975001293
Website19: 0.009999494975001293
Website20: 0.009999494975001293
Website21: 0.009999494975001293
Website22: 0.009999494975001293
Website23: 0.009999494975001293
Website24: 0.009999494975001293
Website25: 0.009999494975001293
Website26: 0.009999494975001293
Website27: 0.009999494975001293
Website28: 0.009999494975001293
Website29: 0.009999494975001293
Website30: 0.009999494975001293
Website31: 0.009999494975001293
Website32: 0.0099

In [134]:
PageRanker.rank(networkAllButOneToAll100, 100, 0.5)
networkAllButOneToAll100.showRanking()
networkAllButOneToAll100.debugRanking()

Website1: 0.01004999747487499
Website2: 0.009999494975001248
Website3: 0.009999494975001248
Website4: 0.009999494975001248
Website5: 0.009999494975001248
Website6: 0.009999494975001248
Website7: 0.009999494975001248
Website8: 0.009999494975001248
Website9: 0.009999494975001248
Website10: 0.009999494975001248
Website11: 0.009999494975001248
Website12: 0.009999494975001248
Website13: 0.009999494975001248
Website14: 0.009999494975001248
Website15: 0.009999494975001248
Website16: 0.009999494975001248
Website17: 0.009999494975001248
Website18: 0.009999494975001248
Website19: 0.009999494975001248
Website20: 0.009999494975001248
Website21: 0.009999494975001248
Website22: 0.009999494975001248
Website23: 0.009999494975001248
Website24: 0.009999494975001248
Website25: 0.009999494975001248
Website26: 0.009999494975001248
Website27: 0.009999494975001248
Website28: 0.009999494975001248
Website29: 0.009999494975001248
Website30: 0.009999494975001248
Website31: 0.009999494975001248
Website32: 0.00999

### Circle

In [116]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")
Website11 = Page("Website11")
Website12 = Page("Website12")
Website13 = Page("Website13")
Website14 = Page("Website14")
Website15 = Page("Website15")
Website16 = Page("Website16")
Website17 = Page("Website17")
Website18 = Page("Website18")
Website19 = Page("Website19")
Website20 = Page("Website20")
Website21 = Page("Website21")
Website22 = Page("Website22")
Website23 = Page("Website23")
Website24 = Page("Website24")
Website25 = Page("Website25")
Website26 = Page("Website26")
Website27 = Page("Website27")
Website28 = Page("Website28")
Website29 = Page("Website29")
Website30 = Page("Website30")
Website31 = Page("Website31")
Website32 = Page("Website32")
Website33 = Page("Website33")
Website34 = Page("Website34")
Website35 = Page("Website35")
Website36 = Page("Website36")
Website37 = Page("Website37")
Website38 = Page("Website38")
Website39 = Page("Website39")
Website40 = Page("Website40")
Website41 = Page("Website41")
Website42 = Page("Website42")
Website43 = Page("Website43")
Website44 = Page("Website44")
Website45 = Page("Website45")
Website46 = Page("Website46")
Website47 = Page("Website47")
Website48 = Page("Website48")
Website49 = Page("Website49")
Website50 = Page("Website50")
Website51 = Page("Website51")
Website52 = Page("Website52")
Website53 = Page("Website53")
Website54 = Page("Website54")
Website55 = Page("Website55")
Website56 = Page("Website56")
Website57 = Page("Website57")
Website58 = Page("Website58")
Website59 = Page("Website59")
Website60 = Page("Website60")
Website61 = Page("Website61")
Website62 = Page("Website62")
Website63 = Page("Website63")
Website64 = Page("Website64")
Website65 = Page("Website65")
Website66 = Page("Website66")
Website67 = Page("Website67")
Website68 = Page("Website68")
Website69 = Page("Website69")
Website70 = Page("Website70")
Website71 = Page("Website71")
Website72 = Page("Website72")
Website73 = Page("Website73")
Website74 = Page("Website74")
Website75 = Page("Website75")
Website76 = Page("Website76")
Website77 = Page("Website77")
Website78 = Page("Website78")
Website79 = Page("Website79")
Website80 = Page("Website80")
Website81 = Page("Website81")
Website82 = Page("Website82")
Website83 = Page("Website83")
Website84 = Page("Website84")
Website85 = Page("Website85")
Website86 = Page("Website86")
Website87 = Page("Website87")
Website88 = Page("Website88")
Website89 = Page("Website89")
Website90 = Page("Website90")
Website91 = Page("Website91")
Website92 = Page("Website92")
Website93 = Page("Website93")
Website94 = Page("Website94")
Website95 = Page("Website95")
Website96 = Page("Website96")
Website97 = Page("Website97")
Website98 = Page("Website98")
Website99 = Page("Website99")
Website100 = Page("Website100")

WebsiteList = [Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
               Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
               Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
               Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
               Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
               Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
               Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
               Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
               Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
               Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100]


i=0
while i<99:
    WebsiteList[i].addLinkTo(WebsiteList[i+1])
    i+=1
Website100.addLinkTo(Website1)


networkCircle100 = Network()
networkCircle100.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
                           Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
                           Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
                           Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
                           Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
                           Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
                           Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
                           Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
                           Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
                           Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100)

In [117]:
PageRanker.rankCR(networkCircle100, 100, 0.5)
networkCircle100.showRanking()
networkCircle100.debugRanking()

Website1: 0.009999999999999941
Website2: 0.009999999999999941
Website3: 0.009999999999999941
Website4: 0.009999999999999941
Website5: 0.009999999999999941
Website6: 0.009999999999999941
Website7: 0.009999999999999941
Website8: 0.009999999999999941
Website9: 0.009999999999999941
Website10: 0.009999999999999941
Website11: 0.009999999999999941
Website12: 0.009999999999999941
Website13: 0.009999999999999941
Website14: 0.009999999999999941
Website15: 0.009999999999999941
Website16: 0.009999999999999941
Website17: 0.009999999999999941
Website18: 0.009999999999999941
Website19: 0.009999999999999941
Website20: 0.009999999999999941
Website21: 0.009999999999999941
Website22: 0.009999999999999941
Website23: 0.009999999999999941
Website24: 0.009999999999999941
Website25: 0.009999999999999941
Website26: 0.009999999999999941
Website27: 0.009999999999999941
Website28: 0.009999999999999941
Website29: 0.009999999999999941
Website30: 0.009999999999999941
Website31: 0.009999999999999941
Website32: 0.0099

In [118]:
PageRanker.rank(networkCircle100, 100, 0.5)
networkCircle100.showRanking()
networkCircle100.debugRanking()

Website1: 0.009999999999999993
Website2: 0.009999999999999993
Website3: 0.009999999999999993
Website4: 0.009999999999999993
Website5: 0.009999999999999993
Website6: 0.009999999999999993
Website7: 0.009999999999999993
Website8: 0.009999999999999993
Website9: 0.009999999999999993
Website10: 0.009999999999999993
Website11: 0.009999999999999993
Website12: 0.009999999999999993
Website13: 0.009999999999999993
Website14: 0.009999999999999993
Website15: 0.009999999999999993
Website16: 0.009999999999999993
Website17: 0.009999999999999993
Website18: 0.009999999999999993
Website19: 0.009999999999999993
Website20: 0.009999999999999993
Website21: 0.009999999999999993
Website22: 0.009999999999999993
Website23: 0.009999999999999993
Website24: 0.009999999999999993
Website25: 0.009999999999999993
Website26: 0.009999999999999993
Website27: 0.009999999999999993
Website28: 0.009999999999999993
Website29: 0.009999999999999993
Website30: 0.009999999999999993
Website31: 0.009999999999999993
Website32: 0.0099

### Cluster

In [141]:
Website1 = Page("Website1")
Website2 = Page("Website2")
Website3 = Page("Website3")
Website4 = Page("Website4")
Website5 = Page("Website5")
Website6 = Page("Website6")
Website7 = Page("Website7")
Website8 = Page("Website8")
Website9 = Page("Website9")
Website10 = Page("Website10")
Website11 = Page("Website11")
Website12 = Page("Website12")
Website13 = Page("Website13")
Website14 = Page("Website14")
Website15 = Page("Website15")
Website16 = Page("Website16")
Website17 = Page("Website17")
Website18 = Page("Website18")
Website19 = Page("Website19")
Website20 = Page("Website20")
Website21 = Page("Website21")
Website22 = Page("Website22")
Website23 = Page("Website23")
Website24 = Page("Website24")
Website25 = Page("Website25")
Website26 = Page("Website26")
Website27 = Page("Website27")
Website28 = Page("Website28")
Website29 = Page("Website29")
Website30 = Page("Website30")
Website31 = Page("Website31")
Website32 = Page("Website32")
Website33 = Page("Website33")
Website34 = Page("Website34")
Website35 = Page("Website35")
Website36 = Page("Website36")
Website37 = Page("Website37")
Website38 = Page("Website38")
Website39 = Page("Website39")
Website40 = Page("Website40")
Website41 = Page("Website41")
Website42 = Page("Website42")
Website43 = Page("Website43")
Website44 = Page("Website44")
Website45 = Page("Website45")
Website46 = Page("Website46")
Website47 = Page("Website47")
Website48 = Page("Website48")
Website49 = Page("Website49")
Website50 = Page("Website50")
Website51 = Page("Website51")
Website52 = Page("Website52")
Website53 = Page("Website53")
Website54 = Page("Website54")
Website55 = Page("Website55")
Website56 = Page("Website56")
Website57 = Page("Website57")
Website58 = Page("Website58")
Website59 = Page("Website59")
Website60 = Page("Website60")
Website61 = Page("Website61")
Website62 = Page("Website62")
Website63 = Page("Website63")
Website64 = Page("Website64")
Website65 = Page("Website65")
Website66 = Page("Website66")
Website67 = Page("Website67")
Website68 = Page("Website68")
Website69 = Page("Website69")
Website70 = Page("Website70")
Website71 = Page("Website71")
Website72 = Page("Website72")
Website73 = Page("Website73")
Website74 = Page("Website74")
Website75 = Page("Website75")
Website76 = Page("Website76")
Website77 = Page("Website77")
Website78 = Page("Website78")
Website79 = Page("Website79")
Website80 = Page("Website80")
Website81 = Page("Website81")
Website82 = Page("Website82")
Website83 = Page("Website83")
Website84 = Page("Website84")
Website85 = Page("Website85")
Website86 = Page("Website86")
Website87 = Page("Website87")
Website88 = Page("Website88")
Website89 = Page("Website89")
Website90 = Page("Website90")
Website91 = Page("Website91")
Website92 = Page("Website92")
Website93 = Page("Website93")
Website94 = Page("Website94")
Website95 = Page("Website95")
Website96 = Page("Website96")
Website97 = Page("Website97")
Website98 = Page("Website98")
Website99 = Page("Website99")
Website100 = Page("Website100")

WebsiteList = [Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
               Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
               Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
               Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
               Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
               Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
               Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
               Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
               Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
               Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100]


i=0
j=0
while i<50:
    while j<50:
        if i != j:
            WebsiteList[i].addLinkTo(WebsiteList[j])
        j+=1
    j=0
    i+=1
i=50
j=50
while i<100:
    while j<100:
        if i != j:
            WebsiteList[i].addLinkTo(WebsiteList[j])
        j+=1
    j=50
    i+=1
    
Website50.addLinkTo(Website51)
Website51.addLinkTo(Website50)



networkCluster100 = Network()
networkCluster100.addPages(Website1,Website2,Website3,Website4,Website5,Website6,Website7,Website8,Website9,Website10,
                           Website11,Website12,Website13,Website14,Website15,Website16,Website17,Website18,Website19,Website20,
                           Website21,Website22,Website23,Website24,Website25,Website26,Website27,Website28,Website29,Website30,
                           Website31,Website32,Website33,Website34,Website35,Website36,Website37,Website38,Website39,Website40,
                           Website41,Website42,Website43,Website44,Website45,Website46,Website47,Website48,Website49,Website50,
                           Website51,Website52,Website53,Website54,Website55,Website56,Website57,Website58,Website59,Website60,
                           Website61,Website62,Website63,Website64,Website65,Website66,Website67,Website68,Website69,Website70,
                           Website71,Website72,Website73,Website74,Website75,Website76,Website77,Website78,Website79,Website80,
                           Website81,Website82,Website83,Website84,Website85,Website86,Website87,Website88,Website89,Website90,
                           Website91,Website92,Website93,Website94,Website95,Website96,Website97,Website98,Website99,Website100)

In [142]:
PageRanker.rankCR(networkCluster100, 100, 0.5)
networkCluster100.showRanking()
networkCluster100.debugRanking()

Website1: 0.009997959600081475
Website2: 0.009997959600081475
Website3: 0.009997959600081475
Website4: 0.009997959600081475
Website5: 0.009997959600081475
Website6: 0.009997959600081475
Website7: 0.009997959600081475
Website8: 0.009997959600081475
Website9: 0.009997959600081475
Website10: 0.009997959600081475
Website11: 0.009997959600081475
Website12: 0.009997959600081475
Website13: 0.009997959600081475
Website14: 0.009997959600081475
Website15: 0.009997959600081475
Website16: 0.009997959600081475
Website17: 0.009997959600081475
Website18: 0.009997959600081475
Website19: 0.009997959600081475
Website20: 0.009997959600081475
Website21: 0.009997959600081475
Website22: 0.009997959600081475
Website23: 0.009997959600081475
Website24: 0.009997959600081475
Website25: 0.009997959600081475
Website26: 0.009997959600081475
Website27: 0.009997959600081475
Website28: 0.009997959600081475
Website29: 0.009997959600081475
Website30: 0.009997959600081475
Website31: 0.009997959600081475
Website32: 0.0099

In [143]:
PageRanker.rank(networkCluster100, 100, 0.5)
networkCluster100.showRanking()
networkCluster100.debugRanking()

Website1: 0.009997959600081601
Website2: 0.009997959600081601
Website3: 0.009997959600081601
Website4: 0.009997959600081601
Website5: 0.009997959600081601
Website6: 0.009997959600081601
Website7: 0.009997959600081601
Website8: 0.009997959600081601
Website9: 0.009997959600081601
Website10: 0.009997959600081601
Website11: 0.009997959600081601
Website12: 0.009997959600081601
Website13: 0.009997959600081601
Website14: 0.009997959600081601
Website15: 0.009997959600081601
Website16: 0.009997959600081601
Website17: 0.009997959600081601
Website18: 0.009997959600081601
Website19: 0.009997959600081601
Website20: 0.009997959600081601
Website21: 0.009997959600081601
Website22: 0.009997959600081601
Website23: 0.009997959600081601
Website24: 0.009997959600081601
Website25: 0.009997959600081601
Website26: 0.009997959600081601
Website27: 0.009997959600081601
Website28: 0.009997959600081601
Website29: 0.009997959600081601
Website30: 0.009997959600081601
Website31: 0.009997959600081601
Website32: 0.0099